<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Chisel Demo
**Next: [Introduction to Scala](1_intro_to_scala.ipynb)**

## Motivation
Perhaps you're an interested student who heard the name "Chisel" tossed about, or maybe you're a seasoned hardware design veteran who has been tasked by your manager to explore Chisel as a new HDL alternative. Either way if you are new to Chisel, you want to figure out as fast as possible what all the fuss is about. Look no futher - let's see what Chisel has to offer!

## Setup
The following cell downloads the dependencies needed for Chisel. You will see it in all future notebooks. **Run this cell now**.

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

These statements are needed to import Chisel. **Run this cell now** before running any future code blocks.

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

# Bringing the Power of Parameterizability and Abstraction to Digital Design

Chisel is powered by [FIRRTL (Flexible Intermediate Representation for RTL)](https://github.com/freechipsproject/firrtl), a hardware compiler framework that performs optimizations of Chisel-generated circuits and supports custom user-defined circuit transformations.

## What does Chisel code look like?

Consider an FIR filter that implements a convolution operation, as depicted in this block diagram:

<img src="https://raw.githubusercontent.com/freechipsproject/chisel3/master/doc/images/fir_filter.svg?sanitize=true" width="512" />

While Chisel provides similar base primitives as synthesizable Verilog, and *could* be used as such:
>The RegNext create registers whose inputs are connected to the argument.

In [ ]:
// 3-point moving average implemented in the style of a FIR filter
class MovingAverage3(bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val z1 = RegNext(io.in)
  val z2 = RegNext(z1)

  io.out := (io.in * 1.U) + (z1 * 1.U) + (z2 * 1.U)
}

By running the previous cell we have created a Scala class. Every time we create an instance of that class we will be generating the hardware that implements the function. Let's take a look at the structure of it

In [ ]:
// same 3-point moving average filter as before
visualize(() => new MovingAverage3(8))

You can see the inputs on the left, and the z1 and z2 registers. Each them and io_in are multiplied by the coefficients and which are then added successively. The tail elements are used to keep the additions from growing.

But the real power of Chisel comes from the ability to create generators, such as n FIR filter that is defined by the list of coefficients. Below we have rewritten MovingAverage to pass into a sequence of coefficients. The number of coefficients will determine the size of the filter.

In [ ]:
// Generalized FIR filter parameterized by the convolution coefficients
class FirFilter(bitWidth: Int, coeffs: Seq[UInt]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })
  // Create the serial-in, parallel-out shift register
  val zs = Reg(Vec(coeffs.length, UInt(bitWidth.W)))
  zs(0) := io.in
  for (i <- 1 until coeffs.length) {
    zs(i) := zs(i-1)
  }

  // Do the multiplies
  val products = VecInit.tabulate(coeffs.length)(i => zs(i) * coeffs(i))

  // Sum up the products
  io.out := products.reduce(_ + _)
}

and use and re-use them across designs:

In [ ]:
// same 3-point moving average filter as before
visualize(() => new FirFilter(8, Seq(1.U, 1.U, 1.U)))
println(getVerilog(new FirFilter(8, Seq(1.U, 1.U, 1.U))))

In [ ]:
// 1-cycle delay as a FIR filter
visualize(() => new FirFilter(8, Seq(0.U, 1.U)))

In [ ]:
// 5-point FIR filter with a triangle impulse response
visualize(() => new FirFilter(8, Seq(1.U, 2.U, 3.U, 2.U, 1.U)))

# Transforming Designs

# Visualizing Designs

---
# You're done!

[Return to the top.](#top)